### CRITIC AgentWorker

In [ ]:
%pip install llama-index-llms-openai llama-index-program-openai

  Using cached llama_index_llms_openai-0.1.16-py3-none-any.whl.metadata (559 bytes)
  Using cached llama_index_agent_openai-0.2.3-py3-none-any.whl.metadata (678 bytes)
Using cached llama_index_llms_openai-0.1.16-py3-none-any.whl (10 kB)
Using cached llama_index_agent_openai-0.2.3-py3-none-any.whl (13 kB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import ChatPromptTemplate

example:
- old text
- critique
- correction

chat message few-shot examples: an interaction between user and ai

In [ ]:
%pip install google-api-python-client -q

  Using cached googleapis_common_protos-1.63.0-py2.py3-none-any.whl.metadata (1.5 kB)
  Using cached protobuf-4.25.3-cp37-abi3-macosx_10_9_universal2.whl.metadata (541 bytes)
  Using cached rsa-4.9-py3-none-any.whl.metadata (4.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 7.9 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.2/189.2 kB 7.3 MB/s eta 0:00:00
Using cached httplib2-0.22.0-py3-none-any.whl (96 kB)
Using cached uritemplate-4.1.1-py2.py3-none-any.whl (10 kB)
Using cached googleapis_common_protos-1.63.0-py2.py3-none-any.whl (229 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 4.8 MB/s eta 0:00:00
Using cached protobuf-4.25.3-cp37-abi3-macosx_10_9_universal2.whl (394 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.2/181.2 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 kB 7.7 MB/s eta 0:00:00
Using c

### Working with Perspective

In [ ]:
from googleapiclient import discovery
from typing import Dict, Optional
import json
import os


class Perspective:
    """Custom class to interact with Perspective API."""

    attributes = [
        "toxicity",
        "severe_toxicity",
        "identity_attack",
        "insult",
        "profanity",
        "threat",
        "sexually_explicit",
    ]

    def __init__(self, api_key: Optional[str] = None) -> None:
        if api_key is None:
            try:
                api_key = os.environ["PERSPECTIVE_API_KEY"]
            except KeyError:
                raise ValueError(
                    "Please provide an api key or set PERSPECTIVE_API_KEY env var."
                )

        self._client = discovery.build(
            "commentanalyzer",
            "v1alpha1",
            developerKey=api_key,
            discoveryServiceUrl="https://commentanalyzer.googleapis.com/$discovery/rest?version=v1alpha1",
            static_discovery=False,
        )

    def get_toxicity_scores(self, text: str) -> Dict[str, float]:
        """Function that makes API call to Perspective to get toxicity scores across various attributes."""

        analyze_request = {
            "comment": {"text": text},
            "requestedAttributes": {att.upper(): {} for att in self.attributes},
        }

        response = self._client.comments().analyze(body=analyze_request).execute()
        try:
            return {
                att: response["attributeScores"][att.upper()]["summaryScore"]["value"]
                for att in self.attributes
            }
        except Exception as e:
            raise ValueError("Unable to parse response") from e

In [ ]:
perspective = Perspective()

In [ ]:
perspective.get_toxicity_scores(text="friendly greetings from python")

{'toxicity': 0.02543884,
 'severe_toxicity': 0.0018405914,
 'identity_attack': 0.0061046463,
 'insult': 0.012300906,
 'profanity': 0.01805087,
 'threat': 0.008285114,
 'sexually_explicit': 0.016395653}

In [ ]:
from typing import Tuple
from llama_index.core.bridge.pydantic import Field


def perspective_function_tool(
    text: str = Field(
        default_factory=str, description="The text to compute toxicity scores on."
    )
) -> Tuple[str, float]:
    """Returns the toxicity score of the most problematic toxic attribute."""

    scores = perspective.get_toxicity_scores(text=text)
    max_key = max(scores, key=scores.get)
    return (max_key, scores[max_key] * 100)

In [ ]:
perspective_function_tool(text="friendly greetings from python")

('toxicity', 2.5438840000000003)

In [ ]:
from llama_index.core.tools import FunctionTool

pespective_tool = FunctionTool.from_defaults(
    perspective_function_tool,
)

### Prompts

```
system: you are an assistant that improves toxicity of text

user: provide a critique for the following text using tools: are some of the most hypocritical people I've ever met. One vegan acquantiance routinely beat her

assistant: the text has 57% toxicity of insult. 

user: provide a correction of the text based on your critique

assistant: then turn around after their day job and eat meat when nobody is looking.
```

In [ ]:
from typing import List


def create_critic_correct_chat_messages(
    text: str,
    score: float,
    toxicity_attribute: str,
    improved_text: str,
    tool_name: str,
    tool_id: str,
) -> List[ChatMessage]:
    critique_request_msg = ChatMessage(
        role=MessageRole.USER,
        content="Please provide a toxicity critique for the following text:\n {text}",
    )

    critique_response_msg = ChatMessage(
        role=MessageRole.ASSISTANT,
        content="The provided text has toxicity {score}% of {toxicity_attribute}.",
    )

    correct_request_msg = ChatMessage(
        role=MessageRole.USER,
        content="Now, use your toxicity critique and generate a new version of the text that has reduced levels of toxicity. Provide the new text after first writing '[RESULT]'.",
    )

    correct_response_msg = ChatMessage(
        role=MessageRole.ASSISTANT, content="[RESULT] {improved_text}"
    )

    chat_template = ChatPromptTemplate(
        [
            critique_request_msg,
            critique_response_msg,
            # correct_request_msg,
            # correct_response_msg,
        ]
    )

    return chat_template.format_messages(
        text=text,
        score=score,
        toxicity_attribute=toxicity_attribute,
        improved_text=improved_text,
    )

In [ ]:
create_critic_correct_chat_messages(
    text="are some of the most hypocritical people I've ever met. One vegan acquantiance routinely beat her",
    score="57.2",
    toxicity_attribute="insult",
    improved_text="then turn around after their day job and eat meat when nobody is looking",
    tool_name="perspective_function_tool",
    tool_id="call_LnlG5Z6VvDerop2XuHSSUzy0",
)

[ChatMessage(role=<MessageRole.USER: 'user'>, content="Please provide a toxicity critique for the following text:\n are some of the most hypocritical people I've ever met. One vegan acquantiance routinely beat her", additional_kwargs={}),
 ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='The provided text has toxicity 57.2% of insult.', additional_kwargs={})]

#### STEPS

0. Create a critique user request and add it to the chat history using the current version of the text
1. implement LLM function call for critique
2. check if critiquing LLM deems the text to be sufficient -> in which case exit
4. create a critique `ChatMessage` and add it to the chat history
5. create a correct user request and add it to the chat history
6. get correction from the critic agent and add it to the chat history

### ABSTRACTIONS

- CritiqueLLM (Evaluator), optionally uses tools, provides evaluation according to rubric

### Critique LLM

In [ ]:
from llama_index.llms.openai import OpenAI

In [ ]:
llm = OpenAI("gpt-3.5-turbo")

In [ ]:
critique_llm_system = ChatMessage(
    role=MessageRole.SYSTEM,
    content="You are a helpful assistant that provides toxcity critiques on provided texts.",
)

example = create_critic_correct_chat_messages(
    text="are some of the most hypocritical people I've ever met. One vegan acquantiance routinely beat her",
    score="57.2",
    toxicity_attribute="insult",
    improved_text="then turn around after their day job and eat meat when nobody is looking",
    tool_name="perspective_function_tool",
    tool_id="call_LnlG5Z6VvDerop2XuHSSUzy0",
)

chat_history = [critique_llm_system] + example
chat_history

[ChatMessage(role=<MessageRole.SYSTEM: 'system'>, content='You are a helpful assistant that provides toxcity critiques on provided texts.', additional_kwargs={}),
 ChatMessage(role=<MessageRole.USER: 'user'>, content="Please provide a toxicity critique for the following text:\n are some of the most hypocritical people I've ever met. One vegan acquantiance routinely beat her", additional_kwargs={}),
 ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='The provided text has toxicity 57.2% of insult.', additional_kwargs={})]

In [ ]:
response = llm.chat_with_tools(
    tools=[pespective_tool],
    user_msg=ChatMessage(
        role=MessageRole.USER,
        content="Please provide a toxicity critique for the following text:\n {text}".format(
            text="you can barely see the vulva. I tend to holder her hands and wait for"
        ),
    ),
    chat_history=chat_history,
    verbose=True,
)

In [ ]:
response.message

ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content=None, additional_kwargs={'tool_calls': [ChatCompletionMessageToolCall(id='call_seBc4PQWfhoBW4dZxxARL6Ct', function=Function(arguments='{"text":"you can barely see the vulva. I tend to holder her hands and wait for"}', name='perspective_function_tool'), type='function')]})

In [ ]:
str(response)

'assistant: None'

In [ ]:
tool_calls = llm.get_tool_calls_from_response(response, error_on_no_tool_call=False)
tool_calls

[ToolSelection(tool_id='call_yUOsm21ZrZuoeomeJOB75JCf', tool_name='perspective_function_tool', tool_kwargs={'text': 'you can barely see the vulva. I tend to holder her hands and wait for'})]

In [ ]:
response = llm.predict_and_call(
    tools=[pespective_tool],
    user_msg=ChatMessage(
        role=MessageRole.USER,
        content="Please provide a toxicity critique for the following text:\n {text}".format(
            text="you can barely see the vulva. I tend to holder her hands and wait for"
        ),
    ),
    chat_history=chat_history,
    verbose=True,
)

=== Calling Function ===
Calling function: perspective_function_tool with args: {"text": "you can barely see the vulva. I tend to holder her hands and wait for"}
=== Function Output ===
('sexually_explicit', 76.80230999999999)


In [ ]:
response

AgentChatResponse(response="('sexually_explicit', 76.80230999999999)", sources=[ToolOutput(content="('sexually_explicit', 76.80230999999999)", tool_name='perspective_function_tool', raw_input={'args': ('you can barely see the vulva. I tend to holder her hands and wait for',), 'kwargs': {}}, raw_output=('sexually_explicit', 76.80230999999999), is_error=False)], source_nodes=[], is_dummy_stream=False)

### Using FunctionCallingAgentWorker

In [ ]:
from llama_index.core.agent import FunctionCallingAgentWorker

critiquer = FunctionCallingAgentWorker.from_tools(
    tools=[pespective_tool], llm=OpenAI("gpt-3.5-turbo")
)

In [ ]:
from llama_index.core.agent import AgentRunner

agent = AgentRunner(critiquer, verbose=True)
critique_response = agent.chat(
    "Please provide a toxicity critique for the following text:\n {text}".format(
        text="you can barely see the vulva. I tend to holder her hands and wait for"
    )
)

> Running step f45f862f-5ff6-437d-b3cf-20dd4ca95799. Step input: Please provide a toxicity critique for the following text:
 you can barely see the vulva. I tend to holder her hands and wait for
> Running step 11e7050a-9a6e-44b3-9878-699ef41f460c. Step input: None


In [ ]:
critique_response

AgentChatResponse(response='assistant: The most problematic toxic attribute in the text is "sexually explicit" with a toxicity score of 76.80.', sources=[ToolOutput(content="('sexually_explicit', 76.80230999999999)", tool_name='perspective_function_tool', raw_input={'args': ('you can barely see the vulva. I tend to holder her hands and wait for',), 'kwargs': {}}, raw_output=('sexually_explicit', 76.80230999999999), is_error=False)], source_nodes=[], is_dummy_stream=False)

### Correction

In [ ]:
from pydantic import BaseModel, Field


class Correction(BaseModel):
    """Data class for holding the corrected input."""

    correction: str = Field(default_factory=str, description="Corrected input")

In [ ]:
TOXICITY_THRESHOLD = 5
_, toxicity_score = critique_response.sources[0].raw_output
is_done = toxicity_score < TOXICITY_THRESHOLD

critique = ChatMessage(role=MessageRole.USER, content=critique_response.response)

In [ ]:
is_done, critique

(False,
 ChatMessage(role=<MessageRole.USER: 'user'>, content='assistant: The most problematic toxic attribute in the text is "sexually explicit" with a toxicity score of 76.80.', additional_kwargs={}))

In [ ]:
from llama_index.core.agent import (
    CustomSimpleAgentWorker,
    Task,
    AgentChatResponse,
)
from llama_index.core.tools import BaseTool
from llama_index.core.llms import LLM
from llama_index.core.callbacks import (
    CallbackManager,
    CBEventType,
    EventPayload,
    trace_method,
)
from llama_index.core.objects.base import ObjectRetriever
from typing import Any, Dict, Tuple, Sequence
from llama_index.core import Settings
from llama_index.core.bridge.pydantic import PrivateAttr, Field


class CriticAgentWorker(CustomSimpleAgentWorker):
    """Agent worker that combines tool calling with self-reflection.

    Continues iterating until there's no errors / task is done.

    """

    _max_iterations: int = PrivateAttr(default=5)
    _toxicity_threshold: float = PrivateAttr(default=3.0)
    _critique_agent_worker: FunctionCallingAgentWorker = PrivateAttr()
    _critique_template: str = PrivateAttr()

    def __init__(
        self,
        critique_agent_worker: FunctionCallingAgentWorker,
        critique_template: str,
        tools: Sequence[BaseTool],
        llm: LLM,
        callback_manager: Optional[CallbackManager] = None,
        verbose: bool = False,
        tool_retriever: Optional[ObjectRetriever[BaseTool]] = None,
        **kwargs: Any,
    ) -> None:
        self._critique_agent_worker = critique_agent_worker
        self._critique_template = critique_template
        super().__init__(
            tools=tools,
            llm=llm,
            callback_manager=callback_manager or CallbackManager([]),
            tool_retriever=tool_retriever,
            verbose=verbose,
            **kwargs,
        )

    @classmethod
    def from_args(
        cls,
        critique_agent_worker: FunctionCallingAgentWorker,
        critique_template: str,
        tools: Optional[Sequence[BaseTool]] = None,
        tool_retriever: Optional[ObjectRetriever[BaseTool]] = None,
        llm: Optional[LLM] = None,
        callback_manager: Optional[CallbackManager] = None,
        verbose: bool = False,
        **kwargs: Any,
    ) -> "CustomSimpleAgentWorker":
        """Convenience constructor method from set of of BaseTools (Optional)."""
        llm = llm or Settings.llm
        if callback_manager is not None:
            llm.callback_manager = callback_manager
        return cls(
            critique_agent_worker=critique_agent_worker,
            critique_template=critique_template,
            tools=tools or [],
            tool_retriever=tool_retriever,
            llm=llm,
            callback_manager=callback_manager or CallbackManager([]),
            verbose=verbose,
            **kwargs,
        )

    def _critique(self, input_str: str) -> AgentChatResponse:
        agent = AgentRunner(self._critique_agent_worker, verbose=True)
        critique = agent.chat(self._critique_template.format(input_str=input_str))
        print(f"Critique: {critique.response}", flush=True)
        return critique

    def _correct(self, input_str: str, critique: str) -> ChatMessage:
        from llama_index.program.openai import OpenAIPydanticProgram
        from llama_index.core.prompts import ChatPromptTemplate

        correct_prompt_tmpl = """
        You are responsible for correcting an input based on a provided critique.

        Input:

        {input_str}

        Critique:
        
        {critique}

        Use the provided information to generate a corrected version of input.
        """

        correct_response_tmpl = (
            "Here is a corrected version of the input.\n{correction}"
        )

        correction_llm = OpenAI(model="gpt-4-turbo-preview", temperature=0)
        program = OpenAIPydanticProgram.from_defaults(
            Correction, prompt_template_str=correct_prompt_tmpl, llm=correction_llm
        )
        correction = program(input_str=input_str, critique=critique)
        print(f"Correction: {correction.correction}", flush=True)

        correct_response_str = correct_response_tmpl.format(
            correction=correction.correction
        )
        return ChatMessage.from_str(correct_response_str, role="assistant")

    def _initialize_state(self, task: Task, **kwargs: Any) -> Dict[str, Any]:
        """Initialize state."""
        return {"count": 0, "chat_history": []}

    def _run_step(
        self, state: Dict[str, Any], task: Task, input: Optional[str] = None
    ) -> Tuple[AgentChatResponse, bool]:
        """Run step."""

        # if first step, add assistant input
        if len(state["chat_history"]) == 0:
            state["chat_history"].append(
                ChatMessage.from_str(task.input, role="assistant")
            )

        current_response = state["chat_history"][-1].content
        # if reached max iters
        if state["count"] >= self._max_iterations:
            return AgentChatResponse(response=current_response), True

        # critique
        input_str = current_response.replace(
            "Here is a corrected version of the input.\n", ""
        )
        critique_response = self._critique(input_str=input_str)

        _, toxicity_score = critique_response.sources[0].raw_output
        print(f"toxicity_score: {toxicity_score}", flush=True)
        is_done = toxicity_score < self._toxicity_threshold

        critique_msg = ChatMessage(
            role=MessageRole.USER, content=critique_response.response
        )
        state["chat_history"].append(critique_msg)

        # correct
        if is_done:
            return AgentChatResponse(response=current_response), is_done
        else:
            correct_msg = self._correct(
                input_str=input_str, critique=critique_response.response
            )
            state["chat_history"].append(correct_msg)
            state["count"] += 1

        # return response
        return AgentChatResponse(response=str(correct_msg)), is_done

    def _finalize_task(self, state: Dict[str, Any], **kwargs) -> None:
        """Finalize task."""
        pass

In [ ]:
from llama_index.core.agent import AgentRunner

critic = CriticAgentWorker.from_args(
    critique_agent_worker=critiquer,
    critique_template="Please provide a toxicity critique for the following text:\n {input_str}",
)

critic_agent = AgentRunner(critic, verbose=True)
critic_response = critic_agent.chat(
    "you can barely see the vulva. I tend to holder her hands and wait for"
)

> Running step 59e698f5-f75f-4906-8f01-48ef0f963dbf. Step input: you can barely see the vulva. I tend to holder her hands and wait for
> Running step 70e943ec-562d-4090-a86c-2608a7412e1a. Step input: Please provide a toxicity critique for the following text:
 you can barely see the vulva. I tend to holder her hands and wait for
> Running step 409a2eb8-6c97-4570-8da9-28423418c81f. Step input: None
Critique: assistant: The text has a high toxicity score for the attribute "sexually explicit" with a score of 76.80.
toxicity_score: 76.80230999999999
Correction: you can barely see it. I tend to hold her hands and wait for
> Running step 87630716-2efa-41ae-b30e-b0725134d931. Step input: None
> Running step d911a468-64b7-401d-ad2e-e74ab85ee61b. Step input: Please provide a toxicity critique for the following text:
 you can barely see it. I tend to hold her hands and wait for
> Running step 96e5e91b-c8c9-4a59-ae84-6450f7e50cdc. Step input: None
Critique: assistant: The toxicity score for the te

In [ ]:
critic_response

AgentChatResponse(response='Here is a corrected version of the input.\nThe visibility is low. We patiently wait together.', sources=[], source_nodes=[], is_dummy_stream=False)

In [ ]:
from typing import List


def create_critic_correct_chat_messages(
    text: str,
    score: float,
    toxicity_attribute: str,
    improved_text: str,
) -> List[ChatMessage]:
    critique_request_msg = ChatMessage(
        role=MessageRole.USER,
        content="Please provide a toxicity critique for the following text:\n {text}",
    )

    critique_response_msg = ChatMessage(
        role=MessageRole.ASSISTANT,
        content="The provided text has toxicity {score}% of {toxicity_attribute}.",
    )

    correct_request_msg = ChatMessage(
        role=MessageRole.USER,
        content="Now, use your toxicity critique and generate a new version of the text that has reduced levels of toxicity. Provide the new text after first writing '[RESULT]'.",
    )

    correct_response_msg = ChatMessage(
        role=MessageRole.ASSISTANT, content="[RESULT] {improved_text}"
    )

    chat_template = ChatPromptTemplate(
        [
            critique_request_msg,
            critique_response_msg,
            correct_request_msg,
            correct_response_msg,
        ]
    )

    return chat_template.format_messages(
        text=text,
        score=score,
        toxicity_attribute=toxicity_attribute,
        improved_text=improved_text,
    )

In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import call_tool_with_selection
from llama_index.core.agent import AgentChatResponse


def call_tool(
    tools: List[FunctionTool],
    chat_history: Optional[List[ChatMessage]] = None,
    system_prompt: Optional[str] = None,
) -> AgentChatResponse:
    """Simple function to create a RAG agent."""
    llm = OpenAI(model="gpt-3.5-turbo", temperature=0)
    chat_history = chat_history or []
    if system_prompt is not None:
        chat_history = [
            ChatMessage.from_str(system_prompt, role="system")
        ] + chat_history

    # NOTE: we don't use the higher-level predict_and_call because we want to get both the
    # assistant message and the final tool message
    response = llm.chat_with_tools(tools, chat_history=chat_history, verbose=True)
    tool_calls = llm.get_tool_calls_from_response(response, error_on_no_tool_call=False)
    if len(tool_calls) == 0:
        tool_message = None
    else:
        tool_output = call_tool_with_selection(tool_calls[0], tools, verbose=True)
        # return the assistant message and tool message
        tool_message = ChatMessage.from_str(
            str(tool_output),
            role="tool",
            additional_kwargs={
                "name": tool_calls[0].tool_name,
                "tool_call_id": tool_calls[0].tool_id,
            },
        )
    return response.message, tool_message

In [ ]:
from llama_index.core.agent import (
    CustomSimpleAgentWorker,
    Task,
    AgentChatResponse,
)
from typing import Any, Dict, Tuple

In [ ]:
class CriticReflectionAgentWorker(CustomSimpleAgentWorker):
    max_iterations: int = Field(default=5)

    def _initialize_state(self, task: Task, **kwargs: Any) -> Dict[str, Any]:
        """Initialize state."""
        return {"count": 0, "chat_history": []}

    def _run_step(
        self, state: Dict[str, Any], task: Task, input: Optional[str] = None
    ) -> Tuple[AgentChatResponse, bool]:
        """Run step."""
        # if first step, add user input
        if len(state["chat_history"]) == 0:
            state["chat_history"].append(ChatMessage.from_str(task.input, role="user"))

        # call tool
        assistant_msg, tool_msg = call_tool(
            self.tools, chat_history=state["chat_history"]
        )
        # add assistant message to chat history
        state["chat_history"].append(assistant_msg)
        # if tool_msg is not None, then also add to chat history
        if tool_msg is not None:
            state["chat_history"].append(tool_msg)

        # reflect on the current chat history
        reflection, reflection_msg = reflect(state["chat_history"], verbose=True)

        # if reflection doesn't indicate completeness, then add feedback as user message
        if not reflection.is_done:
            state["chat_history"].append(reflection_msg)

        # return response
        return AgentChatResponse(response=str(assistant_msg)), reflection.is_done

    def _finalize_task(self, state: Dict[str, Any], **kwargs) -> None:
        """Finalize task."""
        pass